# What kind of SN do the NSNS mergers experience? 

Jeff mentioned many of the Posydon NSNS experience a ECSN, do we see the same? 

In [1]:
import numpy as np
import os 
import sys
import pandas as pd
import h5py as h5
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
import matplotlib.lines as mlines
import multiprocessing as mp

# add run_data path to sys
sys.path.append('./run_data')
from definitions import sim_flags_dict


######################################
## PLOT setttings
plt.rc('font', family='serif')
from matplotlib import rc
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
fsize, SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = 30,20,25,30
for obj in ['axes','xtick','ytick']:
    plt.rc(obj, labelsize=SMALL_SIZE)          # controls default text sizes
for obj in ['figure','axes']:
    plt.rc(obj, titlesize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize



home_dir    = os.path.expanduser("~") 
compas_v    = "v03.01.02" #"v02.46.01" # #"#v02.35.02/"
datar_root  =  f"{home_dir}/ceph/CompasOutput/{compas_v}/"


## Read the potential DCO data 
this table includes all systems that become a DCO at any Z (i.e. more info than just the DCO data)

### and subdivide by DCO flavour


In [2]:
    
def get_merging_dcos(sim_name =  'NewWinds_RemFryer2012', prog_table_name = 'potential_DCO_progenitors_Allinfo.h5'):
    """Read in the potential DCO progenitor table and return the tables for BBH, BHNS and NSNS mergers

    Args:
        sim_name (str, optional): which simulation to load Default 'NewWinds_RemFryer2012'.
        prog_table_name (str, optional): 'potential_DCO_progenitors_Allinfo.h5'.

    Returns:
        BBH_progenitors, BHNS_progenitors, NSNS_progenitors: pandas tables containing the progenitors of BBH, BHNS and NSNS mergers
    """
    ####################################
    # Take the base potential DCO progenitor table
    if os.path.isfile(datar_root+ f'/{sim_name}/'+prog_table_name):
        potential_DCO_progenitors = pd.read_hdf(datar_root + f'{sim_name}/' + prog_table_name, key='All_DCO')
        print('Table exists, reading in... ' ) #potential_DCO_progenitors.info()
    else:
        print('error, table doesnt exist, you should make it using "DataManipulation.ipynb" first')

    potential_DCO_progenitors.reset_index(drop=True, inplace=True)

    # Also make tables that contain only the actual BBH, BHNS and NSNS mergers
    def DCO_merger_bool(table, flavor = 'BBH'):
        """return the DCO merger bool for a given flavor

        Args:
            pandas table: containing all potential DCOs (incl. things that don't become DCO at specific Z)
            flavor (str, optional): BBH, BHNS, or NSNS 
        """
        BBH_bool = np.logical_and(table['Stellar_Type(1)'] == 14,table['Stellar_Type(2)'] == 14)
        BHNS_bool = np.logical_or(np.logical_and(table['Stellar_Type(1)'] == 13,table['Stellar_Type(2)'] == 14),
                                np.logical_and(table['Stellar_Type(1)'] == 14,table['Stellar_Type(2)'] == 13) )
        NSNS_bool = np.logical_and(table['Stellar_Type(1)'] == 13,table['Stellar_Type(2)'] == 13)
        if flavor == 'BBH':
            return np.logical_and(BBH_bool,table['Merges_Hubble_Time'] == True)
        elif flavor == 'BHNS':  
            return np.logical_and(BHNS_bool,table['Merges_Hubble_Time'] == True)
        elif flavor == 'NSNS':
            return np.logical_and(NSNS_bool,table['Merges_Hubble_Time'] == True)

    # make the tables for every flavour
    merging_BBH_bool  = DCO_merger_bool(potential_DCO_progenitors, flavor = 'BBH')
    BBH_progenitors  = potential_DCO_progenitors[merging_BBH_bool]

    merging_BHNS_bool  = DCO_merger_bool(potential_DCO_progenitors, flavor = 'BHNS')
    BHNS_progenitors = potential_DCO_progenitors[merging_BHNS_bool]

    merging_NSNS_bool  = DCO_merger_bool(potential_DCO_progenitors, flavor = 'NSNS')
    NSNS_progenitors = potential_DCO_progenitors[merging_NSNS_bool]

    return [BBH_progenitors, BHNS_progenitors, NSNS_progenitors]





In [5]:
# Read the data
merging_DCO_tables  = get_merging_dcos(sim_name =  'NewWinds_RemFryer2012_oldRSG', prog_table_name = 'potential_DCO_progenitors_Allinfo.h5')

BHBH_progenitors    = merging_DCO_tables[0]

# Select the merging  NSNS that have a CE event
NSNS_progenitors    = merging_DCO_tables[2]
NSNS_progenitors_CE = NSNS_progenitors[NSNS_progenitors['CE_Event_Counter'] > 0]

Table exists, reading in... 


In [6]:
BHBH_progenitors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51877 entries, 280 to 525752
Columns: 111 entries, SEED to SN_star2_Orb_Velocity<SN
dtypes: float64(98), int32(2), object(3), uint32(1), uint64(1), uint8(6)
memory usage: 41.7+ MB


In [5]:

print(np.sum(NSNS_progenitors_CE['Merges_Hubble_Time']), len(NSNS_progenitors_CE))

print(np.unique(NSNS_progenitors_CE['Stellar_Type(1)']), np.unique(NSNS_progenitors_CE['Stellar_Type(2)']) )


10464.0 10464
[13] [13]


In [3]:
sim_name    =  'NewWinds_RemFryer2012_oldRSG' 
channel_key = '_CE'

print('Loading the DCO seeds')
All_DCO_seeds = np.loadtxt(datar_root+ f'/{sim_name}/All_DCO_seeds{channel_key}.txt')

# Open the HDF5 file for all systems at a given metallicity
All_data = h5.File(datar_root+ f'/{sim_name}/COMPAS_Output_combinedZ.h5', 'r')
#################################################################################
# Finally, add supernove information
print('Loading the SN information')
with h5.File(datar_root+f'/{sim_name}/COMPAS_Output_combinedZ.h5', 'r') as All_data:       
    print(All_data['BSE_Supernovae'].keys())

    # Read SN info as pandas dataframes
    SNe = pd.DataFrame()
    
    # Select only the SN events for systems that could potentially become a DCO
    SN_mask = np.in1d(All_data['BSE_Supernovae']['SEED'][()], All_DCO_seeds)

    SN_keys_of_interest = ['SEED', 'Metallicity@ZAMS(1)', 'SN_Type(SN)', 'Supernova_State']
    for key in SN_keys_of_interest:
        read_data   = All_data['BSE_Supernovae'][key][()]
        SNe[key]    = read_data[SN_mask]

    #Add unique seed key
    SNe['unique_Z_SEED'] = [f"{seed}_{Z:.5f}" for seed, Z in zip(SNe['SEED'], SNe['Metallicity@ZAMS(1)'])]



Loading the DCO seeds
Loading the SN information
<KeysViewHDF5 ['Applied_Kick_Magnitude(SN)', 'ComponentSpeed(CP)', 'ComponentSpeed(SN)', 'Drawn_Kick_Magnitude(SN)', 'Eccentricity', 'Eccentricity<SN', 'Experienced_RLOF(SN)', 'Fallback_Fraction(SN)', 'Is_Hydrogen_Poor(SN)', 'Luminosity(CP)', 'Luminosity(SN)', 'MT_Donor_Hist(SN)', 'Mass(CP)', 'Mass(SN)', 'Mass@ZAMS(CP)', 'Mass@ZAMS(SN)', 'Mass_CO_Core@CO(SN)', 'Mass_Core@CO(SN)', 'Mass_He_Core@CO(SN)', 'Mass_Total@CO(SN)', 'Metallicity@ZAMS(1)', 'Orb_Velocity<SN', 'Radius(CP)', 'Radius(SN)', 'Record_Type', 'SEED', 'SN_Kick_Mean_Anomaly(SN)', 'SN_Kick_Phi(SN)', 'SN_Kick_Theta(SN)', 'SN_Orbit_Inclination_Angle', 'SN_Type(SN)', 'SemiMajorAxis', 'SemiMajorAxis<SN', 'Stellar_Type(CP)', 'Stellar_Type(SN)', 'Stellar_Type_Prev(SN)', 'Supernova_State', 'SystemicSpeed', 'Teff(CP)', 'Teff(SN)', 'Time', 'Unbound']>


In [7]:
# SNe[SNe['unique_Z_SEED'] == '4501801_0.01414']
SNe[SNe['SEED']== 4501801]

,SEED,Metallicity@ZAMS(1),SN_Type(SN),Supernova_State,unique_Z_SEED
0,4501801,0.014142,2,1,4501801_0.01414
22292,4501801,0.000548,2,1,4501801_0.00055
67740,4501801,0.001000,2,1,4501801_0.00100
92580,4501801,0.004000,2,1,4501801_0.00400
116037,4501801,0.020000,1,2,4501801_0.02000
137936,4501801,0.002000,2,1,4501801_0.00200
159838,4501801,0.000300,2,1,4501801_0.00030
184235,4501801,0.000173,2,1,4501801_0.00017
207101,4501801,0.010000,2,1,4501801_0.01000
229768,4501801,0.006325,2,1,4501801_0.00632


In [8]:
SN1_types, counts_1 = np.unique(NSNS_progenitors_CE['SN_Type(1)'], return_counts=True)
print('SN1_types', SN1_types, 'counts_1', counts_1)

SN2_types, counts_2 = np.unique(NSNS_progenitors_CE['SN_Type(2)'], return_counts=True)
print('SN2_types', SN2_types, 'counts_2', counts_2)


print(NSNS_progenitors_CE['unique_Z_SEED'][NSNS_progenitors_CE['SN_Type(2)'] == -1])

# print('unique SN type 1', np.unique(potential_NSNS_progenitors['SN_Type(1)']))

# print('unique SN type 2', np.unique(potential_NSNS_progenitors['SN_Type(2)']) )

# potential_NSNS_progenitors[['SN_Type(1)', 'SN_Type(2)']]

SN1_types [-1.  1.  2.] counts_1 [1539 3307 5618]
SN2_types [-1.  1.] counts_2 [10222   242]
16        4501801_0.01414
51        4505173_0.01414
96        4508957_0.01414
162       4515717_0.01414
164       4515785_0.01414
               ...       
559186    2029196_0.00010
559196    2029653_0.00010
559278    2041069_0.00010
559314    2045348_0.00010
559539    4757752_0.00010
Name: unique_Z_SEED, Length: 10222, dtype: object


In [12]:
# SN 1
print('SN1: fraction of ECSN', counts_1[2]/len(NSNS_progenitors_CE))
print('SN1: fraction of CCSN', counts_1[1]/len(NSNS_progenitors_CE))
print('SN1: fraction of err/noSN1?', counts_1[0]/len(NSNS_progenitors_CE))

# SN 2
print('SN2: fraction of CCSN', counts_2[1]/len(NSNS_progenitors_CE))
print('SN2: fraction of err/noSN1?', counts_2[0]/len(NSNS_progenitors_CE))  #???? Why are there systems with SN2 = -1?
# print('SN2: fraction of ECSN', counts_2[2]/len(NSNS_progenitors_CE))




SN1: fraction of ECSN 0.536888379204893
SN1: fraction of CCSN 0.31603593272171254
SN1: fraction of err/noSN1? 0.1470756880733945
SN2: fraction of CCSN 0.02312691131498471
SN2: fraction of err/noSN1? 0.9768730886850153


In [7]:
sim_name    =  'NewWinds_RemFryer2012_oldRSG' 
channel_key = '_CE'

####################################
# Take the base potential DCO progenitor table
prog_table_name = f'potential_DCO_progenitors_Allinfo{channel_key}.h5'

if os.path.isfile(datar_root+ f'/{sim_name}/'+prog_table_name):
    potential_DCO_progenitors = pd.read_hdf(datar_root + f'{sim_name}/' + prog_table_name, key='All_DCO')
    print('Table exists, reading in... ',  potential_DCO_progenitors.info())
else:
    print(f'error, {datar_root}/{sim_name}/{prog_table_name} does not exist, you should make it using "DataManipulation.ipynb" first')

potential_DCO_progenitors.reset_index(drop=True, inplace=True) 


with h5.File(datar_root+f'{sim_name}/COMPAS_Output_combinedZ.h5', 'r') as All_data:
    DCO = All_data['BSE_Double_Compact_Objects']
    st1 = DCO['Stellar_Type(1)'][()]
    st2 = DCO['Stellar_Type(2)'][()]
    dco_merger = DCO['Merges_Hubble_Time'][()]  
    DCO_seed = DCO['SEED'][()]
    # Now I want to add a bool that tells me if this system is ever a BBH, BHNS or BNS progenitor
    BBH_bool = np.logical_and(st1 == 14,st2 == 14)
    BHNS_bool = np.logical_or(np.logical_and(st1 == 13,st2 == 14),
                            np.logical_and(st1 == 14,st2 == 13) )
    NSNS_bool = np.logical_and(st1 == 13,st2 == 13)
    merger_bool = dco_merger == 1

    # Split our potential DCO progenitors into BBH, BHNS and NSNS progenitors
    potential_BBH_progenitors  = potential_DCO_progenitors[np.in1d(potential_DCO_progenitors['SEED'], np.unique(DCO_seed[BBH_bool*merger_bool]) )]
    potential_BHNS_progenitors = potential_DCO_progenitors[np.in1d(potential_DCO_progenitors['SEED'], np.unique(DCO_seed[BHNS_bool*merger_bool]) )]
    potential_NSNS_progenitors = potential_DCO_progenitors[np.in1d(potential_DCO_progenitors['SEED'], np.unique(DCO_seed[NSNS_bool*merger_bool]) )]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 159096 entries, 0 to 159095
Columns: 113 entries, SEED to SN_star2_Orb_Velocity<SN
dtypes: float64(100), int32(2), object(3), uint32(1), uint64(1), uint8(6)
memory usage: 130.2+ MB
Table exists, reading in...  None


In [13]:
list(potential_BBH_progenitors.keys())

['SEED',
 'Metallicity@ZAMS(1)',
 'Stellar_Type(1)',
 'Stellar_Type(2)',
 'CE_Event_Counter',
 'Mass@ZAMS(1)',
 'Mass@ZAMS(2)',
 'SemiMajorAxis@ZAMS',
 'Merger',
 'Merger_At_Birth',
 'Unbound',
 'Immediate_RLOF>CE',
 'Optimistic_CE',
 'Applied_Kick_Magnitude(1)',
 'Applied_Kick_Magnitude(2)',
 'CH_on_MS(1)',
 'SN_Kick_Magnitude_Random_Number(1)',
 'SN_Kick_Phi(1)',
 'SN_Kick_Theta(1)',
 'SN_Kick_Mean_Anomaly(1)',
 'SN_Kick_Magnitude_Random_Number(2)',
 'SN_Kick_Phi(2)',
 'SN_Kick_Theta(2)',
 'SN_Kick_Mean_Anomaly(2)',
 'Merges_Hubble_Time',
 'SemiMajorAxis@DCO',
 'Coalescence_Time',
 'Eccentricity@DCO',
 'MT_Donor_Hist(1)',
 'MT_Donor_Hist(2)',
 'Mass(1)',
 'Mass(2)',
 'unique_Z_SEED',
 'firstMT_SemiMajorAxis<MT',
 'firstMT_SemiMajorAxis>MT',
 'firstMT_Radius(1)<MT',
 'firstMT_Radius(2)<MT',
 'firstMT_Radius(1)>MT',
 'firstMT_Radius(2)>MT',
 'firstMT_Mass(1)<MT',
 'firstMT_Mass(2)<MT',
 'firstMT_Mass(1)>MT',
 'firstMT_Mass(2)>MT',
 'firstMT_Stellar_Type(1)<MT',
 'firstMT_Stellar_Type(2

In [20]:
unbound_and_high_Z = np.logical_and(potential_BBH_progenitors['Metallicity@ZAMS(1)'] == 0.02,potential_BBH_progenitors['Unbound'] == True)
Seed_bool = potential_BBH_progenitors['SEED'] == 79522


In [21]:
potential_BBH_progenitors[['Mass@ZAMS(1)','SemiMajorAxis@ZAMS','firstMT_Stellar_Type(1)<MT','firstMT_SemiMajorAxis<MT', 'SN_star1_Supernova_State', 'SN_star1_Mass_CO_Core@CO(SN)','Applied_Kick_Magnitude(1)','Unbound',]][unbound_and_high_Z]


,Mass@ZAMS(1),SemiMajorAxis@ZAMS,firstMT_Stellar_Type(1)<MT,firstMT_SemiMajorAxis<MT,SN_star1_Supernova_State,SN_star1_Mass_CO_Core@CO(SN),Applied_Kick_Magnitude(1),Unbound
